# LLM-Powered Word Sense Disambiguation

This notebook helps you automatically identify the correct **WordNet sense** (synset) for ambiguous words in context using Large Language Models.

---

## 🚀 Getting Started

### Step 1: Save Your Own Copy

**Important!** You're viewing a shared notebook. To run it and save your work:

1. Click **File** → **Save a copy in Drive**
2. A new tab will open with your personal copy
3. You can now edit, run cells, and your changes will be saved automatically

> 💡 **Tip:** Rename your copy (click the title at the top) to something memorable like "WSD Annotation - My Project"

### Step 2: Connect to a Runtime

Before running any code, you need to connect to Google's servers:

1. Click **Connect** in the top-right corner (or **Runtime** → **Connect**)
2. Wait for the green checkmark ✓
3. You now have access to a virtual machine that will run your code

> ⚠️ **Note:** Free Colab sessions disconnect after ~90 minutes of inactivity. Your saved notebook won't be lost, but you'll need to re-run cells from the beginning.

---

## 📋 What This Notebook Does

1. **Takes your input**: A word, its sentence context, and candidate senses
2. **Queries an LLM**: Sends the disambiguation task to a language model
3. **Returns the answer**: The model selects the most appropriate WordNet synset
4. **Evaluates results**: If you provide ground truth labels, calculates accuracy

### Example Task

| Component | Example |
|-----------|---------|
| **Word** | *bank* |
| **Context** | "She walked along the river bank at sunset." |
| **Candidates** | `08420278-n` (financial institution), `09213565-n` (sloping land) |
| **Model Output** | `09213565-n` ✓ |

---

## 🔧 Requirements

- A Google account (for Colab)
- An API key from your chosen provider (OpenAI, DeepSeek, OpenRouter, etc.)
- Your annotation data (words, contexts, and candidate synsets)

Let's get started! Run each cell in order by clicking the ▶️ play button or pressing `Shift+Enter`.

---
## 1️⃣ Setup & Installation

Run this cell to install required libraries and download WordNet data. This may take 1-2 minutes.

In [2]:
#@title 🔧 Install Libraries & Download WordNet { display-mode: "form" }
#@markdown Click the ▶️ button to run this cell. You only need to do this once per session.

import subprocess
import sys

print("📦 Installing required libraries...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "wn", "openai"])

print("📚 Downloading Open English WordNet (this may take a moment)...")
import wn
try:
    wn.download("oewn:2024")
except wn.Error:
    print("   WordNet already downloaded!")

print("\n✅ Setup complete!")

📦 Installing required libraries...
📚 Downloading Open English WordNet (this may take a moment)...


Download [##############################] (12912118/12912118 bytes) Complete
Read [##############################] (1298444/1298444) 
Added oewn:2024 (Open English Wordnet)



✅ Setup complete!


---
## 2️⃣ API Configuration

### What is an API?

An **API** (Application Programming Interface) is a way for programs to communicate with each other. In this notebook, we use APIs to send text to language models (like GPT-4 or Claude) and receive their responses.

Think of it like ordering at a restaurant:
- You (the notebook) send a request to the kitchen (the API)
- The kitchen processes your order using their recipes (the model)
- You receive your meal (the model's response)

### What is an API Key?

An **API key** is like a password that:
- **Identifies you** to the service provider
- **Tracks your usage** for billing purposes
- **Keeps your requests secure**

> 🔒 **Security Note:** Your API key is entered securely (hidden as you type) and stays only in your browser session. It's never saved to the notebook file or sent anywhere except to your chosen API provider.

---

### 🔑 How to Get an API Key

Choose your provider and follow the instructions:

<details>
<summary><b>OpenAI (GPT-4, GPT-4o, etc.)</b></summary>

1. Go to [platform.openai.com](https://platform.openai.com/)
2. Sign up or log in
3. Click your profile icon → **View API keys**
4. Click **Create new secret key**
5. Copy the key immediately (you won't see it again!)
6. Add credits at **Billing** → **Add payment method**

**Cost:** ~$2.50–10 per 1M input tokens depending on model. For most WSD tasks, expect to spend pennies to a few dollars.

</details>

<details>
<summary><b>DeepSeek</b></summary>

1. Go to [platform.deepseek.com](https://platform.deepseek.com/)
2. Sign up or log in
3. Navigate to **API Keys** in your dashboard
4. Create a new key and copy it

**Cost:** Very affordable; check current pricing on their website.

</details>

<details>
<summary><b>OpenRouter (access to many models)</b></summary>

1. Go to [openrouter.ai](https://openrouter.ai/)
2. Sign up or log in (you can use Google/GitHub)
3. Go to **Keys** in the menu
4. Click **Create Key**
5. Copy your key

**Benefit:** One API key gives you access to models from OpenAI, Anthropic, Google, Meta, Mistral, and more. Some models are even free!

**Model names:** Use format like `openai/gpt-4o`, `anthropic/claude-3.5-sonnet`, `google/gemini-2.0-flash-exp:free`

</details>

<details>
<summary><b>Custom / Self-hosted</b></summary>

If you're running your own model server (e.g., with vLLM, Ollama, or text-generation-inference), enter your server's URL. The endpoint should be OpenAI-compatible.

Example: `http://localhost:8000/v1` or `https://your-server.com/v1`

</details>

---

### 💰 How Much Will This Cost?

API pricing is based on **tokens** (roughly 4 characters = 1 token). A typical WSD task might use:
- ~200–500 tokens per annotation (depending on your prompt and candidate list)
- 100 annotations ≈ 20,000–50,000 tokens

| Provider | Model | Approximate Cost for 100 Annotations |
|----------|-------|--------------------------------------|
| OpenAI | gpt-4o-mini | ~0.01–0.03 |
| OpenAI | gpt-4o | ~0.05–0.15 |
| DeepSeek | deepseek-chat | ~0.01–0.02 |
| OpenRouter | Various free models | 0.00 |

> 💡 **Tip:** Start with a smaller batch (5–10 items) to test your setup before running large annotation jobs.

In [3]:
#@title 🌐 Select API Provider { display-mode: "form" }

api_provider = "DeepSeek" #@param ["OpenAI", "DeepSeek", "OpenRouter", "Custom"]

#@markdown ---
#@markdown **Custom Base URL** (only used if "Custom" is selected above):
custom_base_url = "https://api.example.com/v1" #@param {type:"string"}

# Define base URLs for each provider
PROVIDER_URLS = {
    "OpenAI": "https://api.openai.com/v1",
    "DeepSeek": "https://api.deepseek.com",
    "OpenRouter": "https://openrouter.ai/api/v1",
}

# Set the base URL
if api_provider == "Custom":
    BASE_URL = custom_base_url
else:
    BASE_URL = PROVIDER_URLS[api_provider]

print(f"🌐 API Provider: {api_provider}")
print(f"🔗 Base URL: {BASE_URL}")

# Store for later use
API_CONFIG = {
    "provider": api_provider,
    "base_url": BASE_URL
}

🌐 API Provider: DeepSeek
🔗 Base URL: https://api.deepseek.com


In [4]:
#@title 🔑 Enter Your API Key { display-mode: "form" }
#@markdown Your key stays private and is not stored anywhere.

import getpass
import os
from openai import OpenAI

# Determine which environment variable to use/check
env_var_name = f"{API_CONFIG['provider'].upper()}_API_KEY"
if API_CONFIG['provider'] == "Custom":
    env_var_name = "CUSTOM_API_KEY"

print(f"🔐 Configuring API key for {API_CONFIG['provider']}...")

# Check if already set
if env_var_name in os.environ and os.environ[env_var_name]:
    api_key = os.environ[env_var_name]
    print(f"✅ Using existing {env_var_name}")
else:
    api_key = getpass.getpass(f"Enter your {API_CONFIG['provider']} API key: ")
    os.environ[env_var_name] = api_key
    print(f"✅ API key saved to {env_var_name} for this session!")

# Initialize the OpenAI client with the configured base URL
client = OpenAI(
    api_key=api_key,
    base_url=API_CONFIG['base_url']
)

print(f"\n✅ Client initialized!")
print(f"   Provider: {API_CONFIG['provider']}")
print(f"   Base URL: {API_CONFIG['base_url']}")

🔐 Configuring API key for DeepSeek...
Enter your DeepSeek API key: ··········
✅ API key saved to DEEPSEEK_API_KEY for this session!

✅ Client initialized!
   Provider: DeepSeek
   Base URL: https://api.deepseek.com


---
## 3️⃣ Model Settings

#### Temperature

**Temperature** controls how "creative" or "random" the model's responses are:

| Value | Behavior | Use For |
|-------|----------|---------|
| **0.0** | Deterministic — same input gives same output | ✅ Annotation tasks |
| **0.3–0.7** | Balanced — some variety | Creative writing |
| **1.0+** | High randomness — very creative | Brainstorming |

For word sense disambiguation, we use **a low temperature** because we want consistent, reproducible results.

#### Reasoning Effort (OpenAI only)

For OpenAI's reasoning models, this controls how much "thinking" the model does:

- **none** — Standard mode (for non-reasoning models)
- **low** — Quick reasoning
- **medium** — Balanced
- **high** — Deep reasoning (slower, more expensive, potentially more accurate)

#### Responses API vs Chat Completions

- **Responses API** — OpenAI's newer API format (supports reasoning models)
- **Chat Completions** — Standard format supported by most providers

> 💡 **Tip:** If you're using DeepSeek, OpenRouter, or a custom provider, uncheck "Use Responses API" to use the compatible Chat Completions format.

In [5]:
#@title ⚙️ Model Configuration { display-mode: "form" }

#@markdown ### Select Model
#@markdown Common models by provider:
#@markdown - **OpenAI**: `gpt-5.2`, `gpt-5.2-mini`, `gpt-5.2-nano` `gpt-4o`, `gpt-4o-mini`, `gpt-4-turbo`, `gpt-3.5-turbo`
#@markdown - **DeepSeek**: `deepseek-chat`, `deepseek-reasoner`
#@markdown - **OpenRouter**: `openai/gpt-4o`, `anthropic/claude-3.5-sonnet`, `google/gemini-2.0-flash-exp:free`

model_name = "deepseek-chat" #@param {type:"string"}

#@markdown ---
#@markdown ### Reasoning Effort
#@markdown (Applies to OpenAI reasoning models; may be ignored by other providers)
reasoning_effort = "none" #@param ["none", "low", "medium", "high"]

#@markdown ---
#@markdown ### Temperature
temperature = 0.0 #@param {type:"number"}

#@markdown ---
#@markdown ### Use Responses API
#@markdown Uncheck for providers that only support Chat Completions API
use_responses_api = False #@param {type:"boolean"}

# Store settings
MODEL_CONFIG = {
    "model": model_name,
    "reasoning_effort": reasoning_effort,
    "use_responses_api": use_responses_api
}

print(f"📊 Model Configuration:")
print(f"   • Provider: {API_CONFIG['provider']}")
print(f"   • Model: {model_name}")
print(f"   • Reasoning effort: {reasoning_effort}")
print(f"   • API: {'Responses' if use_responses_api else 'Chat Completions'}")
print("\n✅ Settings saved!")

📊 Model Configuration:
   • Provider: DeepSeek
   • Model: deepseek-chat
   • Reasoning effort: none
   • API: Chat Completions

✅ Settings saved!


---
## 3️⃣ Custom System Prompt

Customize the instructions given to the model. This prompt defines how the model should approach the word sense disambiguation task.

The prompt can use these placeholders (automatically filled in for each task):
- `{token}` — The word to disambiguate
- `{context}` — The sentence containing the word  
- `{synset_list}` — Formatted list of candidate synsets with definitions

**Tip:** You can use Markdown formatting in your prompt.

In [6]:
#@title 📝 Configure System Prompt { display-mode: "form" }

#@markdown ### System Instructions
#@markdown These instructions set the model's role and behavior. Click on the leftmost arrow to see and modify them.

system_prompt = """You are an expert linguistic annotator specializing in word sense disambiguation (WSD). Your task is to identify the correct WordNet synset for a target word based on its context.

## Your Approach

1. **Read the context carefully** — understand how the target word is being used
2. **Consider each candidate synset** — review the definition and example lemmas
3. **Select the best match** — choose the synset that most accurately captures the intended meaning

## Important Guidelines

- Focus on the **semantic meaning** in context, not just surface similarity
- Consider **domain and register** — is this technical, colloquial, figurative?
- When multiple synsets seem plausible, choose the **most specific** one that fits
- If the word is used **metaphorically**, consider whether a literal or figurative sense applies

## Response Format

Respond with **ONLY** the synset ID (e.g., `08420278-n`). Do not include any explanation, reasoning, or additional text."""

#@markdown ---
#@markdown ### User Prompt Template
#@markdown This template is filled in for each annotation task. Use `{token}`, `{context}`, and `{synset_list}` as placeholders. Click on the leftmost arrow to see and modify the template.

user_prompt_template = """**TARGET WORD:** {token}

**CONTEXT:** {context}

**CANDIDATE SYNSETS:**
{synset_list}

Which synset ID best matches the meaning of "{token}" in the given context?"""

# Store prompts
PROMPT_CONFIG = {
    "system": system_prompt,
    "user_template": user_prompt_template
}

print("📝 Prompt Configuration Saved!")
print("\n" + "=" * 70)
print("SYSTEM PROMPT PREVIEW:")
print("=" * 70)
print(system_prompt[:500] + "..." if len(system_prompt) > 500 else system_prompt)
print("\n" + "=" * 70)
print("USER PROMPT TEMPLATE PREVIEW:")
print("=" * 70)
print(user_prompt_template)

📝 Prompt Configuration Saved!

SYSTEM PROMPT PREVIEW:
You are an expert linguistic annotator specializing in word sense disambiguation (WSD). Your task is to identify the correct WordNet synset for a target word based on its context.

## Your Approach

1. **Read the context carefully** — understand how the target word is being used
2. **Consider each candidate synset** — review the definition and example lemmas
3. **Select the best match** — choose the synset that most accurately captures the intended meaning

## Important Guidelines

- Focus on th...

USER PROMPT TEMPLATE PREVIEW:
**TARGET WORD:** {token}

**CONTEXT:** {context}

**CANDIDATE SYNSETS:**
{synset_list}

Which synset ID best matches the meaning of "{token}" in the given context?


---
## 4️⃣ Input Your Data

Choose how you want to enter your annotation data:

- **Option A: Text Input** — Type or paste directly (best for 1-10 items)
- **Option B: File Upload** — Upload a CSV or JSON file (best for larger datasets)

In [7]:
#@title 📝 Choose Input Method { display-mode: "form" }

input_method = "CSV Upload" #@param ["Text Input", "CSV Upload", "JSON Upload"]

print(f"📋 Selected: {input_method}")

if input_method == "Text Input":
    print("\n👇 Run the next cell to enter your data in a text box.")
elif input_method == "CSV Upload":
    print("\n👇 Run the CSV upload cell below.")
else:
    print("\n👇 Run the JSON upload cell below.")

📋 Selected: CSV Upload

👇 Run the CSV upload cell below.


---
### Option A: Text Input

Enter your data in the text box below using this format:

token: bank  
context: I walked along the river bank.  
candidates: bank, shore, 08420278-n  
ground_truth: 08420278-n

token: run  
context: The program will run overnight.  
candidates: run, execute, operate

**Field explanations:**
- `token` — The word to disambiguate
- `context` — The sentence containing the word
- `candidates` — Comma-separated list of lemmas (like `bank`) and/or synset offsets (like `08420278-n`)
- `ground_truth` — (Optional) The correct synset offset for evaluation

Use `---` to separate multiple entries.

In [ ]:
#@title ✏️ Enter Your Data Here { display-mode: "form" }

#@markdown Click on the leftmost arrow to show the code and paste your data in the text_input field. Follow thes format shown above.

text_input = """token: bank
context: I deposited money at the bank this morning.
candidates: bank, financial_institution, shore, river
ground_truth: 08420278-n
---
token: bank
context: We had a picnic on the river bank.
candidates: bank, financial_institution, shore, slope
ground_truth: 09213565-n
---
token: bright
context: She is a bright student who excels in mathematics.
candidates: bright, intelligent, luminous, shining
"""

# Parse the text input
def parse_text_input(text):
    """Parse the structured text format into a list of annotation tasks."""
    entries = []
    current_entry = {}

    for line in text.strip().split('\n'):
        line = line.strip()

        if line == '---':
            if current_entry:
                entries.append(current_entry)
                current_entry = {}
            continue

        if ':' in line:
            key, value = line.split(':', 1)
            key = key.strip().lower()
            value = value.strip()

            if key == 'candidates':
                # Split candidates and clean whitespace
                value = [c.strip() for c in value.split(',')]

            current_entry[key] = value

    # Don't forget the last entry
    if current_entry:
        entries.append(current_entry)

    return entries

# Only process if Text Input was selected
if input_method == "Text Input":
    annotation_data = parse_text_input(text_input)
    print(f"✅ Parsed {len(annotation_data)} annotation task(s):\n")

    for i, entry in enumerate(annotation_data, 1):
        print(f"📌 Task {i}:")
        print(f"   Token: {entry.get('token', '❌ MISSING')}")
        print(f"   Context: {entry.get('context', '❌ MISSING')[:60]}...")
        print(f"   Candidates: {entry.get('candidates', '❌ MISSING')}")
        if 'ground_truth' in entry:
            print(f"   Ground truth: {entry['ground_truth']}")
        print()
else:
    print("⏭️ Skipping text input — you selected file upload.")
    annotation_data = []

In [8]:
#@title 📄 Upload CSV File { display-mode: "form" }
#@markdown **CSV Format:** Your file should have these columns:
#@markdown - `token` — the word to annotate
#@markdown - `context` — the sentence
#@markdown - `candidates` — comma-separated lemmas/offsets (in one cell)
#@markdown - `ground_truth` — (optional) correct synset offset

import csv
import io

if input_method == "CSV Upload":
    from google.colab import files

    print("📤 Click 'Choose Files' to upload your CSV:")
    uploaded = files.upload()

    for filename, content in uploaded.items():
        print(f"\n📂 Processing: {filename}")
        decoded = content.decode('utf-8')
        reader = csv.DictReader(io.StringIO(decoded))

        annotation_data = []
        for row in reader:
            entry = {
                'token': row.get('token', '').strip(),
                'context': row.get('context', '').strip(),
                'candidates': [c.strip() for c in row.get('candidates', '').split(',')],
            }
            if row.get('ground_truth', '').strip():
                entry['ground_truth'] = row['ground_truth'].strip()
            annotation_data.append(entry)

        print(f"✅ Loaded {len(annotation_data)} annotation task(s)")
        break  # Only process first file

else:
    print("⏭️ Skipping CSV upload — you selected a different input method.")

📤 Click 'Choose Files' to upload your CSV:


Saving example_ground_truth.csv to example_ground_truth.csv

📂 Processing: example_ground_truth.csv
✅ Loaded 4 annotation task(s)


In [ ]:
#@title 📄 Upload JSON File { display-mode: "form" }
#@markdown **JSON Format:**
#@markdown ```json
#@markdown [
#@markdown   {
#@markdown     "token": "bank",
#@markdown     "context": "I walked along the river bank.",
#@markdown     "candidates": ["bank", "shore", "08420278-n"],
#@markdown     "ground_truth": "08420278-n"
#@markdown   }
#@markdown ]
#@markdown ```

import json

if input_method == "JSON Upload":
    from google.colab import files

    print("📤 Click 'Choose Files' to upload your JSON:")
    uploaded = files.upload()

    for filename, content in uploaded.items():
        print(f"\n📂 Processing: {filename}")
        annotation_data = json.loads(content.decode('utf-8'))
        print(f"✅ Loaded {len(annotation_data)} annotation task(s)")
        break  # Only process first file

else:
    print("⏭️ Skipping JSON upload — you selected a different input method.")

---
## 5️⃣ Resolve Candidate Synsets

This step expands your candidates into full WordNet synsets:
- **Lemmas** (like `bank`) → All matching synsets are retrieved
- **Synset offsets** (like `08420278-n`) → Looked up directly

You'll see each synset's definition so you can verify the candidates make sense.

In [11]:
#@title 🔍 Resolve Synsets from Candidates { display-mode: "form" }

import wn
import re

# Initialize English WordNet
oewn = wn.Wordnet('oewn:2024')

def is_synset_offset(s):
    """Check if string looks like a synset offset (e.g., '08420278-n')."""
    return bool(re.match(r'^\d{8}-[nvarsp]$', s))

def resolve_candidates(candidates):
    """
    Convert a list of lemmas and/or synset offsets into synset objects.
    Returns a dict mapping synset IDs to synset info.
    """
    resolved = {}

    for candidate in candidates:
        candidate = candidate.strip()

        if is_synset_offset(candidate):
            # It's a synset offset — look it up directly
            try:
                # Construct the full synset ID for ewn
                synset_id = f"oewn-{candidate}"
                ss = wn.synset(synset_id)
                resolved[candidate] = {
                    'synset_id': candidate,
                    'definition': ss.definition(),
                    'lemmas': ss.lemmas(),
                    'source': f"offset:{candidate}"
                }
            except wn.Error:
                print(f"   ⚠️ Could not find synset: {candidate}")

        else:
            # It's a lemma — find all synsets
            synsets = oewn.synsets(candidate)
            for ss in synsets:
                # Extract offset from synset ID (e.g., 'ewn-08420278-n' → '08420278-n')
                offset = ss.id.replace('oewn-', '') # Changed ss.id() to ss.id
                if offset not in resolved:
                    resolved[offset] = {
                        'synset_id': offset,
                        'definition': ss.definition(),
                        'lemmas': ss.lemmas(),
                        'source': f"lemma:{candidate}"
                    }

    return resolved

# Process all annotation tasks
print("🔍 Resolving synsets for all tasks...\n")
print("=" * 70)

for i, entry in enumerate(annotation_data, 1):
    print(f"\n📌 Task {i}: '{entry['token']}' in \"{entry['context'][:50]}...\"")
    print("-" * 70)

    resolved = resolve_candidates(entry['candidates'])
    entry['resolved_synsets'] = resolved

    if not resolved:
        print("   ❌ No synsets found! Check your candidates.")
        continue

    print(f"   Found {len(resolved)} unique synset(s):\n")

    for syn_id, info in resolved.items():
        lemma_str = ', '.join(info['lemmas'][:5])
        if len(info['lemmas']) > 5:
            lemma_str += '...'
        print(f"   🔹 {syn_id}")
        print(f"      Lemmas: {lemma_str}")
        print(f"      Definition: {info['definition'][:80]}...")
        print()

print("=" * 70)
print(f"\n✅ Synset resolution complete for {len(annotation_data)} task(s)!")

🔍 Resolving synsets for all tasks...


📌 Task 1: 'invident' in "existimat. haec qui gaudent, gaudeant perpetuo suo..."
----------------------------------------------------------------------
   Found 4 unique synset(s):

   🔹 07565182-n
      Lemmas: envy, enviousness
      Definition: a feeling of grudging admiration and desire to have something that is possessed ...

   🔹 00759688-n
      Lemmas: envy, invidia
      Definition: spite and resentment at seeing the success of another (personified as one of the...

   🔹 01831561-v
      Lemmas: envy
      Definition: feel envious towards; admire enviously...

   🔹 01831006-v
      Lemmas: envy, begrudge
      Definition: be envious of; set one's heart on...


📌 Task 2: 'ἐξακούω' in "φρένας. ποῦ ποῦ καθίζωμʼ ἐν καλῷ, τῶν ῥητόρων ἵνʼ ..."
----------------------------------------------------------------------
   Found 6 unique synset(s):

   🔹 02174146-v
      Lemmas: hear
      Definition: perceive (sound) via the auditory sense...

   🔹 00

---
## 6️⃣ Run Annotation

Now we'll send each token + context to the language model, asking it to choose the most appropriate synset from the candidates.

The model receives:
- The token to disambiguate
- The context sentence
- A list of candidate synsets with their definitions

It returns the synset ID it believes is correct.

In [12]:
#@title 🚀 Run Semantic Annotation { display-mode: "form" }

import json
import time

def build_prompt(token, context, synsets_info):
    """Build the user prompt using the configured template."""

    synset_descriptions = []
    for syn_id, info in synsets_info.items():
        lemmas = ', '.join(info['lemmas'][:5])
        synset_descriptions.append(
            f"- **{syn_id}**: {info['definition']} _(lemmas: {lemmas})_"
        )

    synset_list = '\n'.join(synset_descriptions)

    # Fill in the template
    prompt = PROMPT_CONFIG['user_template'].format(
        token=token,
        context=context,
        synset_list=synset_list
    )

    return prompt


def annotate_single(token, context, synsets_info, model_config):
    """Send a single annotation request to the API."""

    user_prompt = build_prompt(token, context, synsets_info)
    synset_ids = list(synsets_info.keys())

    try:
        if model_config.get('use_responses_api', True):
            # Use Responses API (OpenAI)
            reasoning_config = None
            if model_config['reasoning_effort'] != "none":
                reasoning_config = {"effort": model_config['reasoning_effort']}

            response = client.responses.create(
                model=model_config['model'],
                instructions=PROMPT_CONFIG['system'],
                input=user_prompt,
                reasoning=reasoning_config
            )
            result = response.output_text.strip()
        else:
            # Use Chat Completions API (compatible with most providers)
            response = client.chat.completions.create(
                model=model_config['model'],
                messages=[
                    {"role": "system", "content": PROMPT_CONFIG['system']},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=temperature  # Deterministic for annotation
            )
            result = response.choices[0].message.content.strip()

        # Clean up the result (remove quotes, backticks, whitespace)
        result = result.strip('`"\'').strip()

        # Validate that result is one of the candidates
        if result in synset_ids:
            return result, "success", None
        else:
            # Try to find a partial match (model might have added extra text)
            for syn_id in synset_ids:
                if syn_id in result:
                    return syn_id, "partial_match", f"Extracted '{syn_id}' from '{result}'"
            return result, "invalid", f"Response '{result}' not in candidates"

    except Exception as e:
        return None, "error", str(e)


# Run annotation for all tasks
print("🚀 Starting annotation...\n")
print(f"   Provider: {API_CONFIG['provider']}")
print(f"   Model: {MODEL_CONFIG['model']}")
print(f"   API: {'Responses' if MODEL_CONFIG.get('use_responses_api', True) else 'Chat Completions'}")
print("\n" + "=" * 70)

results = []

for i, entry in enumerate(annotation_data, 1):
    token = entry['token']
    context = entry['context']
    synsets = entry.get('resolved_synsets', {})

    print(f"\n📌 Task {i}/{len(annotation_data)}: '{token}'")
    print(f"   Context: \"{context[:60]}...\"")
    print(f"   Candidates: {len(synsets)} synset(s)")

    if not synsets:
        print("   ⚠️ Skipping — no resolved synsets")
        results.append({
            'token': token,
            'context': context,
            'predicted': None,
            'status': 'skipped',
            'message': 'No synsets resolved'
        })
        continue

    # Call the API
    predicted, status, message = annotate_single(token, context, synsets, MODEL_CONFIG)

    result_entry = {
        'token': token,
        'context': context,
        'num_candidates': len(synsets),
        'candidates': list(synsets.keys()),
        'predicted': predicted,
        'status': status
    }

    if 'ground_truth' in entry:
        result_entry['ground_truth'] = entry['ground_truth']
        result_entry['correct'] = (predicted == entry['ground_truth'])

    if message:
        result_entry['message'] = message

    results.append(result_entry)

    # Display result
    if status == "success":
        print(f"   ✅ Predicted: {predicted}")
        if 'ground_truth' in entry:
            if result_entry['correct']:
                print(f"   🎯 Correct! (ground truth: {entry['ground_truth']})")
            else:
                print(f"   ❌ Incorrect (ground truth: {entry['ground_truth']})")
    elif status == "partial_match":
        print(f"   ⚠️ Predicted: {predicted} ({message})")
    else:
        print(f"   ❌ Error: {message}")

    # Small delay to avoid rate limiting
    time.sleep(0.5)

print("\n" + "=" * 70)
print(f"\n✅ Annotation complete! Processed {len(results)} task(s).")

# Store results globally
annotation_results = results

🚀 Starting annotation...

   Provider: DeepSeek
   Model: deepseek-chat
   API: Chat Completions


📌 Task 1/4: 'invident'
   Context: "existimat. haec qui gaudent, gaudeant perpetuo suo semper bo..."
   Candidates: 4 synset(s)
   ✅ Predicted: 01831561-v
   🎯 Correct! (ground truth: 01831561-v)

📌 Task 2/4: 'ἐξακούω'
   Context: "φρένας. ποῦ ποῦ καθίζωμʼ ἐν καλῷ, τῶν ῥητόρων ἵνʼ ἐξακούω; σ..."
   Candidates: 6 synset(s)
   ✅ Predicted: 02175483-v
   ❌ Incorrect (ground truth: 02193614-v)

📌 Task 3/4: 'praevidit'
   Context: "et alte extulit: ille ictum venientem a vertice velox praevi..."
   Candidates: 4 synset(s)
   ✅ Predicted: 02133754-v
   ❌ Incorrect (ground truth: 00722732-v)

📌 Task 4/4: 'ἐσιδεῖν'
   Context: "πᾷ ποτε τῶνδε πόνων χρή σε τέρμα κέλ - σαντʼ ἐσιδεῖν · ἀκίχη..."
   Candidates: 2 synset(s)
   ✅ Predicted: 02133754-v
   ❌ Incorrect (ground truth: 00592510-v)


✅ Annotation complete! Processed 4 task(s).


---
## 7️⃣ Results & Evaluation

View your annotation results and evaluation metrics below.

In [13]:
#@title 📊 View Results { display-mode: "form" }

print("📊 ANNOTATION RESULTS")
print("=" * 70)

# Display each result
for i, result in enumerate(annotation_results, 1):
    print(f"\n📌 Task {i}: '{result['token']}'")
    print(f"   Context: {result['context'][:60]}...")
    print(f"   Candidates: {result['num_candidates']} synset(s)")
    print(f"   Predicted: {result.get('predicted', 'N/A')}")

    if 'ground_truth' in result:
        status = "✅ CORRECT" if result.get('correct') else "❌ INCORRECT"
        print(f"   Ground truth: {result['ground_truth']}")
        print(f"   Status: {status}")

print("\n" + "=" * 70)

# Summary statistics
total = len(annotation_results)
successful = sum(1 for r in annotation_results if r['status'] == 'success')

print(f"\n📈 SUMMARY:")
print(f"   Total tasks: {total}")
print(f"   Successfully annotated: {successful}")
print(f"   Errors/skipped: {total - successful}")

📊 ANNOTATION RESULTS

📌 Task 1: 'invident'
   Context: existimat. haec qui gaudent, gaudeant perpetuo suo semper bo...
   Candidates: 4 synset(s)
   Predicted: 01831561-v
   Ground truth: 01831561-v
   Status: ✅ CORRECT

📌 Task 2: 'ἐξακούω'
   Context: φρένας. ποῦ ποῦ καθίζωμʼ ἐν καλῷ, τῶν ῥητόρων ἵνʼ ἐξακούω; σ...
   Candidates: 6 synset(s)
   Predicted: 02175483-v
   Ground truth: 02193614-v
   Status: ❌ INCORRECT

📌 Task 3: 'praevidit'
   Context: et alte extulit: ille ictum venientem a vertice velox praevi...
   Candidates: 4 synset(s)
   Predicted: 02133754-v
   Ground truth: 00722732-v
   Status: ❌ INCORRECT

📌 Task 4: 'ἐσιδεῖν'
   Context: πᾷ ποτε τῶνδε πόνων χρή σε τέρμα κέλ - σαντʼ ἐσιδεῖν · ἀκίχη...
   Candidates: 2 synset(s)
   Predicted: 02133754-v
   Ground truth: 00592510-v
   Status: ❌ INCORRECT


📈 SUMMARY:
   Total tasks: 4
   Successfully annotated: 4
   Errors/skipped: 0


In [14]:
#@title 📏 Evaluation Metrics { display-mode: "form" }
#@markdown This cell calculates accuracy if ground truth labels were provided.

# Check if we have ground truth
has_ground_truth = any('ground_truth' in r for r in annotation_results)

if not has_ground_truth:
    print("ℹ️ No ground truth labels provided — skipping evaluation.")
else:
    print("📏 EVALUATION METRICS")
    print("=" * 70)

    # Filter to only tasks with ground truth
    eval_tasks = [r for r in annotation_results if 'ground_truth' in r and r['status'] == 'success']

    if not eval_tasks:
        print("⚠️ No successfully annotated tasks with ground truth.")
    else:
        # Calculate accuracy
        correct = sum(1 for r in eval_tasks if r.get('correct', False))
        total = len(eval_tasks)
        accuracy = correct / total * 100

        print(f"\n🎯 ACCURACY: {correct}/{total} = {accuracy:.1f}%")

        # Calculate average number of candidates (for context)
        avg_candidates = sum(r['num_candidates'] for r in eval_tasks) / len(eval_tasks)
        random_baseline = 100 / avg_candidates

        print(f"\n📊 CONTEXT:")
        print(f"   Average candidates per task: {avg_candidates:.1f}")
        print(f"   Random baseline accuracy: {random_baseline:.1f}%")
        print(f"   Improvement over random: {accuracy - random_baseline:+.1f} percentage points")

        # Breakdown by number of candidates
        print(f"\n📋 BREAKDOWN BY DIFFICULTY:")

        from collections import defaultdict
        by_num_candidates = defaultdict(list)
        for r in eval_tasks:
            by_num_candidates[r['num_candidates']].append(r.get('correct', False))

        for num_cands in sorted(by_num_candidates.keys()):
            tasks = by_num_candidates[num_cands]
            acc = sum(tasks) / len(tasks) * 100
            print(f"   {num_cands} candidates: {sum(tasks)}/{len(tasks)} correct ({acc:.1f}%)")

        print("\n" + "=" * 70)

📏 EVALUATION METRICS

🎯 ACCURACY: 1/4 = 25.0%

📊 CONTEXT:
   Average candidates per task: 4.0
   Random baseline accuracy: 25.0%
   Improvement over random: +0.0 percentage points

📋 BREAKDOWN BY DIFFICULTY:
   2 candidates: 0/1 correct (0.0%)
   4 candidates: 1/2 correct (50.0%)
   6 candidates: 0/1 correct (0.0%)



---
## 8️⃣ Export Results

Download your annotation results as JSON or CSV.

In [15]:
#@title 💾 Export Results { display-mode: "form" }

export_format = "CSV" #@param ["JSON", "CSV"]

from google.colab import files
import json
import csv
import io

if export_format == "JSON":
    # Export as JSON
    filename = "annotation_results.json"
    content = json.dumps(annotation_results, indent=2, ensure_ascii=False)

    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)

    print(f"📄 JSON Preview:\n")
    print(content[:500] + "..." if len(content) > 500 else content)

else:
    # Export as CSV
    filename = "annotation_results.csv"

    fieldnames = ['token', 'context', 'num_candidates', 'predicted', 'ground_truth', 'correct', 'status']

    output = io.StringIO()
    writer = csv.DictWriter(output, fieldnames=fieldnames, extrasaction='ignore')
    writer.writeheader()
    for result in annotation_results:
        writer.writerow(result)

    content = output.getvalue()

    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)

    print(f"📄 CSV Preview:\n")
    print(content[:500] + "..." if len(content) > 500 else content)

print(f"\n" + "=" * 70)
print(f"\n⬇️ Downloading {filename}...")
files.download(filename)

📄 CSV Preview:

token,context,num_candidates,predicted,ground_truth,correct,status
invident,"existimat. haec qui gaudent, gaudeant perpetuo suo semper bono; qui invident, ne umquam eorum quisquam invideat prosus commodis. Age accumbe",4,01831561-v,01831561-v,True,success
ἐξακούω,"φρένας. ποῦ ποῦ καθίζωμʼ ἐν καλῷ, τῶν ῥητόρων ἵνʼ ἐξακούω; σὺ δʼ ἄπιθʼ ὦ Θρᾷττʼ ἐκποδών. δούλοις γὰρ οὐκ",6,02175483-v,02193614-v,False,success
praevidit,"et alte extulit: ille ictum venientem a vertice velox praevidit, celerique el...


⬇️ Downloading annotation_results.csv...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
## 📎 Appendix: Download Templates

Run the cells below to download blank templates for CSV or JSON input.

In [ ]:
#@title 📥 Download CSV Template { display-mode: "form" }

from google.colab import files

csv_template = """token,context,candidates,ground_truth
bank,"I deposited money at the bank this morning.","bank, financial_institution, shore",08420278-n
bank,"We had a picnic on the river bank.","bank, shore, slope",09213565-n
bright,"She is a bright student.","bright, intelligent, luminous",
"""

with open('annotation_template.csv', 'w') as f:
    f.write(csv_template)

print("📄 CSV Template Preview:")
print("-" * 50)
print(csv_template)
print("-" * 50)
print("\n⬇️ Downloading template...")
files.download('annotation_template.csv')

In [ ]:
#@title 📥 Download JSON Template { display-mode: "form" }

from google.colab import files
import json

json_template = [
    {
        "token": "bank",
        "context": "I deposited money at the bank this morning.",
        "candidates": ["bank", "financial_institution", "shore", "08420278-n"],
        "ground_truth": "08420278-n"
    },
    {
        "token": "bank",
        "context": "We had a picnic on the river bank.",
        "candidates": ["bank", "shore", "slope"],
        "ground_truth": "09213565-n"
    },
    {
        "token": "bright",
        "context": "She is a bright student who excels in mathematics.",
        "candidates": ["bright", "intelligent", "luminous", "shining"]
    }
]

content = json.dumps(json_template, indent=2)

with open('annotation_template.json', 'w') as f:
    f.write(content)

print("📄 JSON Template Preview:")
print("-" * 50)
print(content)
print("-" * 50)
print("\n⬇️ Downloading template...")
files.download('annotation_template.json')